# Import package

In [1]:
#Install module
# %pip install --user --upgrade pandas
%pip install --user SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import package
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import requests
from sqlalchemy import BigInteger, Numeric, Date, Text, String
import time
# import os
# from tqdm import tqdm
# from datetime import datetime
# import yfinance as yf
# import lxml
# from bs4 import BeautifulSoup #BeutifulSoup

# Data Preparation

# Some useful functions

In [2]:
#Get Alpha Vantage api key
def get_apikey(filename: str):
    with open(filename) as f:
        api_key = f.read().strip()
    f.close
    return api_key

In [3]:
#Get daily core stock data from Alpha Vantage from 2000-01 to now
def get_daily_stockdata(ticker:str, outputsize = 'compact', datatype = 'json') -> pd.DataFrame:
    function = 'TIME_SERIES_DAILY'
    datatype = datatype
    outputsize = outputsize
    alpha_vantage_apikey = get_apikey(filename= 'dist/apikey_AlphaVantage')

    url = f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&outputsize={outputsize}&apikey={alpha_vantage_apikey}&datatype={datatype}'
    r = requests.get(url)
    df = pd.DataFrame.from_dict(r.json()['Time Series (Daily)'],orient='index')
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = df.rename_axis('Date').reset_index()
    df.insert(loc = 0, column = 'Ticker', value = ticker, allow_duplicates=True)
    return df

In [4]:
def get_company_overview(ticker:str) -> dict:
    #Delay api call
    time.sleep(15)
    function = 'OVERVIEW'
    alpha_vantage_apikey = get_apikey(filename= 'dist/apikey_AlphaVantage')

    url = f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&apikey={alpha_vantage_apikey}'
    r = requests.get(url)
    return r.json()

## Web Scrapping

This section will perform web scrapping to scrap all S&P500 tickers on wikipedia. https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

We can simply use padnas to get the table from wiki.

In [6]:
#Store the S&P information in pandas dataframe
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
tickers_df_list = pd.read_html(wiki_url)
tickers_df = tickers_df_list[0]
tickers_df.head()
industry_dimension = tickers_df.loc[:, ['Symbol','GICS Sector','GICS Sub-Industry']]

In [7]:
industry_dimension

,Symbol,GICS Sector,GICS Sub-Industry
0,MMM,Industrials,Industrial Conglomerates
1,AOS,Industrials,Building Products
2,ABT,Health Care,Health Care Equipment
3,ABBV,Health Care,Pharmaceuticals
4,ACN,Information Technology,IT Consulting & Other Services
...,...,...,...
498,YUM,Consumer Discretionary,Restaurants
499,ZBRA,Information Technology,Electronic Equipment & Instruments
500,ZBH,Health Care,Health Care Equipment
501,ZION,Financials,Regional Banks


In [8]:
#Turn pandas dataframe to the list
tickers = tickers_df['Symbol'].values.tolist()
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'C

## Download the financial data

In [5]:
#Download the financial data from Alpha vantage
# tickers_test = ['AAPL','MMM','ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL']

stock_df = pd.DataFrame(columns=['Ticker','Date','Open', 'High', 'Low', 'Close', 'Volume'])
for ticker in tickers:
    try: stock_df = pd.concat([stock_df, get_daily_stockdata(ticker,outputsize='full')], ignore_index = True)
    except: continue
stock_df.tail()

NameError: name 'tickers' is not defined

In [ ]:
#Download the company overview from Alpha vantage

#Define the columns
temp_json = get_company_overview('IBM')
columns = [key for key in temp_json.keys()]

#Create a new dict to store the stock overview data
StockOverview_dict = {}
for column in columns:
    StockOverview_dict[column] = []

#Append the value from api result
for ticker in tickers:
    try: r = get_company_overview(ticker)
    except:continue
    for key, value in r.items():
        try: StockOverview_dict[key].append(value)
        except:
            StockOverview_dict[key] = []
            StockOverview_dict[key].append(value)

#Convert the dict to pandas dataframe
StockOverview_df = pd.DataFrame.from_dict(StockOverview_dict)
StockOverview_df.tail()

In [14]:
StockOverview_dict

{'Symbol': ['MMM',
  'AOS',
  'ABT',
  'ABBV',
  'BIIB',
  'BLK',
  'BK',
  'BA',
  'BKNG',
  'BWA',
  'BXP',
  'BSX',
  'GILD',
  'GL',
  'GPN',
  'GS',
  'HAL',
  'HIG',
  'HAS',
  'PRU',
  'PEG',
  'PTC',
  'PSA',
  'PHM',
  'QRVO',
  'PWR'],
 'AssetType': ['Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock',
  'Common Stock'],
 'Name': ['3M Company',
  'Smith AO Corporation',
  'Abbott Laboratories',
  'AbbVie Inc',
  'Biogen Inc',
  'BlackRock Inc',
  'Bank of New York Mellon',
  'The Boeing Company',
  'Booking Holdings Inc',
  'BorgWarner Inc',
  'Boston Properties Inc',
  'Boston Scientific

In [ ]:
#Connect SQLite
import sqlite3

conn = sqlite3.connect('StockData.db')
stock_cursor = conn.cursor()

In [ ]:
#Store the data into SQLite3 database
StockPrice_tableName = 'StockPrice'
StockOverview_tableName = 'StockOverview'

#Create a table called, StockPrice, in SQLite3
stock_df.to_sql(StockPrice_tableName,conn,if_exists='replace',
                index=True,index_label='Record')
StockOverview_df.to_sql(StockOverview_tableName,conn,if_exists='replace',
                        index=True,index_label='Record')
conn.commit()


In [ ]:
#Select all column and limit to show top 100
sql = '''
SELECT * 
FROM StockOverview
LIMIT 100;  
'''

test = pd.read_sql(sql,conn)
test

,Record,Ticker,Date,Open,High,Low,Close,Volume
0,0,MMM,2023-07-28,111.8700,112.4100,111.0450,111.8800,2910759
1,1,MMM,2023-07-27,110.4500,113.1400,110.4500,111.1900,5613488
2,2,MMM,2023-07-26,109.8400,113.0700,109.8400,112.6400,5594986
3,3,MMM,2023-07-25,107.5500,110.8500,107.0092,109.8300,8778880
4,4,MMM,2023-07-24,104.5300,105.6041,103.9800,104.2700,3333380
...,...,...,...,...,...,...,...,...
95,95,MMM,2023-03-13,103.6700,104.6800,102.7300,103.5000,4211092
96,96,MMM,2023-03-10,105.5400,107.0000,103.7400,104.0600,3693856
97,97,MMM,2023-03-09,107.6600,108.5600,105.4200,105.5200,3463725
98,98,MMM,2023-03-08,107.4400,107.7700,106.1800,107.1600,3434665
